Note: Please change the URLs for the reference csv files. Then, delete the code block that is marked

In [ ]:
from villas.dataprocessing.readtools import *
from villas.dataprocessing.timeseries import *
from villas.dataprocessing.timeseries import TimeSeries as ts
import matplotlib.pyplot as plt
import re
import os
import urllib.request

# %matplotlib widget

# DP Simulation of topology with slack, line and VSI

## DP simulation

In [ ]:
%%bash
TOP=${TOP:-$(git rev-parse --show-toplevel)}
PATH=${TOP}/../build/dpsim/Examples/Cxx

DURATION=8.0
TIMESTEP=3e-3

DP_Slack_PiLine_VSI_Ramp_with_PF_Init --timestep=${TIMESTEP} --duration=${DURATION}

In [ ]:
modelName = 'DP_Slack_PiLine_VSI_Ramp_with_PF_Init_DP'
path = 'logs/' + modelName + '/'
dpsim_result_file = path + modelName + '.csv'

ts_dpsim = read_timeseries_csv(dpsim_result_file)

## DP and EMT simulation for reference

In [ ]:
if not os.path.exists('reference-results'):
    os.mkdir('reference-results')

In [ ]:
# Adjust reference file path here (step 1/3)
url = 'https://git.rwth-aachen.de/acs/public/simulation/reference-results/raw/master/.../DP_Slack_PiLine_VSI_Ramp_with_PF_Init_DP.csv'
dp_ref_file = 'reference-results/DP_Slack_PiLine_VSI_Ramp_with_PF_Init_DP.csv'
try:
    urllib.request.urlretrieve(url, dp_ref_file) 
    print(dp_ref_file)
    ts_dpsim_ref_dp = read_timeseries_csv(dpsim_result_file)
    dp_reference_available = True
except:
    dp_reference_available = False

In [ ]:
# Adjust reference file path here (step 2/3)
url = 'https://git.rwth-aachen.de/acs/public/simulation/reference-results/raw/master/.../EMT_Slack_PiLine_VSI_Ramp_with_PF_Init_EMT.csv'
emt_ref_file = 'reference-results/EMT_Slack_PiLine_VSI_Ramp_with_PF_Init_EMT.csv'
try:
    urllib.request.urlretrieve(url, emt_ref_file) 
    print(emt_ref_file)
    ts_dpsim_ref_emt = read_timeseries_csv(dpsim_result_file)
    emt_reference_available = True
except:
    emt_reference_available = False

In [ ]:
# Delete these four lines from the notebook (step 3/3)
ts_dpsim_ref_dp = read_timeseries_csv('~/rsa-git/DP_Slack_PiLine_VSI_Ramp_with_PF_Init_DP.csv')
dp_reference_available = True
ts_dpsim_ref_emt = read_timeseries_csv('~/rsa-git/EMT_Slack_PiLine_VSI_Ramp_with_PF_Init_EMT.csv')
emt_reference_available = True

## Plots

In [ ]:
plt.figure(figsize=(12,6))
ts_obj = ts_dpsim['f_src']
plt.plot(ts_obj.time, ts_obj.values, label='f_src', color='C0')

plt.xlim(4.8,8)
plt.xlabel('Zeit [s]')
plt.ylabel('Frequenz der Spannungsquelle [Hz]')
plt.gcf().legend(loc='upper center', ncol=2)
plt.show()

In [ ]:
plt.figure(figsize=(12,6))
if emt_reference_available:
    ts_obj = ts_dpsim_ref_emt['v2_0']
    plt.plot(ts_obj.time, np.sqrt(3/2)*ts_obj.values, label='u2 (EMT, 50 µs ref)')
if dp_reference_available:
    ts_obj = ts_dpsim_ref_dp['v2']
    plt.plot(ts_obj.time, ts_obj.abs().values, label='u2 abs (DP, 50 µs ref)', color='C1')
ts_obj = ts_dpsim['v2']
plt.plot(ts_obj.time, ts_obj.abs().values, label='u2 abs (DP, 1 ms)', color='C1', linestyle='--')

plt.xlim(4.8,8)
plt.ylim(20014,20030)
plt.xlabel('Time [s]')
plt.ylabel('Load voltage magnitude v2 [V]')
plt.gcf().legend(loc='upper center', ncol=3)
plt.grid('y')
plt.show()

In [ ]:
plt.figure(figsize=(12,6))
if emt_reference_available:
    ts_obj = ts_dpsim_ref_emt['v2_0']
    plt.plot(ts_obj.time, np.sqrt(3/2)*ts_obj.values, label='u2 (EMT, 50 µs ref)')

ts_v2_interpolated = ts_dpsim['v2'].abs().interpolate(50e-6)
ts_v2_shifted = ts.frequency_shift(ts_v2_interpolated, 50)
plt.plot(ts_v2_shifted.time, ts_v2_shifted.values, label='u2 (DP, 1 µs interpolated, shifted)', color='C1')

plt.xlim(4.8,8)
#plt.xlim(6.0,6.1)
plt.ylim(20014,20030)
plt.xlabel('Time [s]')
plt.ylabel('Load voltage magnitude [V]')
plt.gcf().legend(loc='upper center', ncol=3)
plt.grid('y')
plt.show()

### Assertion

In [ ]:
if emt_reference_available:
    error_abs = np.absolute(np.sqrt(3/2)*ts_dpsim_ref_emt['v2_0'].values[80000:] - ts_obj_shifted.values[80000:160002]).max()
    print('EMT ref v2_0 vs. DP v2_shift (abs): ' + str(error_abs))
    assert error_abs < 10

In [ ]:
if dp_reference_available:
    error_abs = np.absolute(ts_dpsim_ref_dp['v2'].values[80000:] - ts_v2_interpolated.values[80000:160002]).max()
    print('DP ref v2 vs. DP v2 (abs): ' + str(error_abs))
    assert error_abs < 10